In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import mne
import os
from collections import Counter
import scipy
import pandas as pd
    
def get_trigger_marker_info_xml(filepath_trigger_markers, filepath_instrument_markers, constrain_recording_time=True, start_id=0):
    #first read the instrument marker
    instrument_marker = get_instrument_marker_information(filepath_instrument_markers)
    date_of_instrument_marker_file = filepath_instrument_markers.split('InstrumentMarker')[-1][:-4]
    time_string = date_of_instrument_marker_file[8:10] + ":" + date_of_instrument_marker_file[10:12]+ ":" + date_of_instrument_marker_file[12:14] + "." + date_of_instrument_marker_file[14:17]
    instrument_marker_time = convert_time_to_ms(time_string,type_of_time=0)

    tree_triggers = ET.parse(filepath_trigger_markers)
    date_of_trigger_markers_file = filepath_trigger_markers.split('_Coil')[-1][2:-4]
    if not date_of_instrument_marker_file[:8] == date_of_trigger_markers_file[:8]: #then we have the same yyyy:mm:dd for both files and we can compare times
        print(f"Different dates in instrument markers and trigger markers!\nCheck files {filepath_trigger_markers} and {filepath_instrument_markers} again!")
        return False

    root_triggers = tree_triggers.getroot()
    starting_time = root_triggers.get('startTime') # in hours:minutes:seconds:milliseconds, e.g. 12:24:06.283 starts at 24 minutes, 6 seconds and 283 milliseconds after 12 pm (noon).
    starting_time_in_ms = convert_time_to_ms(starting_time,type_of_time=0) #starting time of the marker file in milliseconds in day time
    coordinate_space_triggers = root_triggers.get('coordinateSpace') #coordinate space of the recording. Should be LPS or RAS.
    bad_times = []
    trigger_markers = [] #initialize list for saving the trigger markers
    for trigger_marker in root_triggers.findall(".//TriggerMarker"): #go through all trigger markers
        recording_time_latency = trigger_marker.get('recordingTime') #get the recording time of the marker
        recording_time = starting_time_in_ms + int(recording_time_latency)
        if instrument_marker_time < recording_time and constrain_recording_time or (not constrain_recording_time): #constrain the recording time to take only into account those stimuli after which the target did not change.
            matrix_element = trigger_marker.find('Matrix4D') #get the coil information of that trigger marker
            if matrix_element is not None: #save the info on the coil position, direction, and orientation
                trigger_info = read_localite_matrix4d(matrix_element)
                trigger_info['recording_time_latency'] = recording_time_latency #save the time of stimulation relative to the start of the recording
                trigger_info['recording_time_in_real_world'] = recording_time
                if coordinate_space_triggers =='LPS': #transform everything into ras
                    trigger_info = transform_from_lps_to_ras(trigger_info)
                    coordinate_space_triggers_now = 'RAS' #new coordinate space
                elif coordinate_space_triggers =='RAS':
                    coordinate_space_triggers_now = 'RAS'
                else:
                    print("Bad coordinate system in trigger markers! Should be LPS or RAS.")
                    return False
                trigger_info = add_matrix4d(trigger_info)
                trigger_info['stimulus_id'] = start_id + len(trigger_markers)
                #trigger_info = calculate_pcd_and_dir_diff(trigger_info, instrument_marker)
                if coordinate_space_triggers_now == 'RAS':
                    trigger_markers.append(trigger_info)
                else:
                    print("Bad coordinate system in trigger markers! Should be LPS or RAS.")
                    return False
        elif constrain_recording_time and instrument_marker_time > recording_time:
            bad_times.append(recording_time)
    if constrain_recording_time and len(bad_times) > 0:
        print(f'Did not save {len(bad_times)} stimuli because the instrument marker was modified after that they were delivered.')
    print(f'Stimuli read: {len(trigger_markers)}.')

    #add time information to the instrument marker too
    instrument_marker['recording_time'] = instrument_marker_time
    instrument_marker['recording_date'] = date_of_instrument_marker_file
    information = {'trigger_markers':trigger_markers, 'instrument_marker':instrument_marker, 'starting_time_of_trigger_marker_recording':starting_time}
    return information

def convert_time_to_ms(time_string, type_of_time):
    if type_of_time == 0:
        #assumes format hours:minutes:seconds.milliseconds
        hours, minutes, seconds_milliseconds =  time_string.split(":")
        seconds, milliseconds = seconds_milliseconds.split(".")
        time_in_ms = (int(hours)*60*60*1000) + (int(minutes)*60*1000) + (int(seconds)*1000) + int(milliseconds)
    else:
        #assumes format hours:minutes:seconds
        hours, minutes, seconds =  time_string.split(":")
        time_in_ms = (int(hours)*60*60*1000) + (int(minutes)*60*1000) + (int(seconds)*1000)
    return time_in_ms


def get_instrument_marker_information(filepath_instrument_markers):
    tree_instruments = ET.parse(filepath_instrument_markers)
    root_instruments = tree_instruments.getroot()
    coordinate_space_instruments = root_instruments.get('coordinateSpace') #Should be LPS or RAS.
    selected_instrument_markers = []
    for instrument_marker_candidate in root_instruments.findall(".//InstrumentMarker[@selected='true']"):
        matrix_element = instrument_marker_candidate.find('.//Matrix4D')
        if matrix_element is not None:
            instrument_marker = read_localite_matrix4d(matrix_element)
            if coordinate_space_instruments == 'LPS': #transform from lps to ras
                instrument_marker = transform_from_lps_to_ras(instrument_marker)
                coordinate_space_instruments = 'RAS' #new coordinate space
            instrument_marker = add_matrix4d(instrument_marker)
            if coordinate_space_instruments == 'RAS':
                selected_instrument_markers.append(instrument_marker)
            else:
                print("Bad coordinate system in instrument markers! Should be LPS or RAS.")
                return False
    if len(selected_instrument_markers) != 1: #there should only be one selected instrument marker
        print("More or less than one selected instrument marker. Check again!")
        return False
    else:
        return instrument_marker
    

def read_localite_matrix4d(matrix_element):
    # see the localite documentation for more
    data = {name: np.float64(value) for name, value in matrix_element.attrib.items()}
    marker_info = {}
    marker_info['coil_normal'] = np.array([data['data00'],data['data10'],data['data20']])
    marker_info['coil_dir'] = np.array([data['data01'],data['data11'],data['data21']])
    marker_info['coil_ori'] = np.array([data['data02'],data['data12'],data['data22']])
    marker_info['coil_pos'] = np.array([data['data03'],data['data13'],data['data23']])
    return marker_info

def add_matrix4d(marker_info):
    #add the matrix to the marker info
    marker_info['matrix'] = np.eye(4)
    marker_info['matrix'][0:3,0] = marker_info['coil_normal']
    marker_info['matrix'][0:3,1] = marker_info['coil_dir']
    marker_info['matrix'][0:3,2] = marker_info['coil_ori']
    marker_info['matrix'][0:3,3] = marker_info['coil_pos']
    return marker_info


def transform_from_lps_to_ras(information_dict):
    #transform from lps to ras (switch order of x and y coordinates)
    transformation_matrix = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]])
    for spatial_key in ['coil_pos', 'coil_ori', 'coil_normal','coil_dir']: #transform the features to RAS that have spatial information in them
        information_dict[spatial_key] = np.matmul(transformation_matrix,information_dict[spatial_key])
    return information_dict #return the modified dictionary


def get_instrument_marker_filepath(subject_localite_path):
    insturment_marker_folder_path = os.path.join(subject_localite_path, 'InstrumentMarkers')
    files_in_instument_marker_folder = os.listdir(insturment_marker_folder_path)
    #get the instrument marker files
    identifier = 'InstrumentMarker'
    instrument_marker_files = [file for file in files_in_instument_marker_folder if file.startswith(identifier)]
    #get the most recent file (compare dates) that are e.g. "InstrumentMarker20240210114243703.xml"
    most_recent_file = max(instrument_marker_files, key=lambda file:file[len(identifier):])
    #define the filepath and read the digitization file
    instrument_marker_filepath = os.path.join(insturment_marker_folder_path,most_recent_file)
    return instrument_marker_filepath


def get_trigger_marker_filepath(subject_localite_path):
    trigger_marker_folder_path = os.path.join(subject_localite_path, 'TMSTrigger')
    try:
        files_in_trigger_marker_folder = os.listdir(trigger_marker_folder_path)
        #get the instrument marker files
        identifier = 'TriggerMarkers'
        trigger_marker_files = [file for file in files_in_trigger_marker_folder if file.startswith(identifier)]
        all_dates = [int(file.split("_Coil")[-1][2:10]) for file in trigger_marker_files]
        latest_date = np.max(all_dates)
        #get the largest file that is in the group of the latest date
        trigger_marker_filepaths = [os.path.join(trigger_marker_folder_path,trigger_marker_file) for trigger_marker_file in trigger_marker_files if str(latest_date) in trigger_marker_file and "Coil1" in trigger_marker_file]
        trigger_markers_filepaths = [fp for fp in trigger_marker_filepaths if os.path.getsize(fp) > 2000]
        sorted_trigger_markers_filepaths = sorted(trigger_markers_filepaths, key=lambda x: int(x[-21:-4]))
        return sorted_trigger_markers_filepaths
    except FileNotFoundError:
        print(f"No TMSTrigger folder exists in {subject_localite_path}")
        return False


def get_trigger_marker_info_nbe(filepath, subject):
    file = open(filepath,"r",encoding= 'unicode_escape')
    lines = file.readlines()
    sequence_starts = []
    line_index = 0
    trigger_markers = []
    target_pos_mte = False
    while line_index < len(lines): #go through all lines
        line = lines[line_index]
        if "Sequence Created" in line and "series" not in line: #observe stimulation sequences
            line_splitted =  line.split(" ")
            date_of_squence = line_splitted[2] + "T"
            time_of_sequence = line_splitted[3].strip("\n")
            sequence_starts.append(date_of_squence + time_of_sequence)
            #get time of the start of the sequence in milliseconds of day time
            time_of_sequence_in_ms = convert_time_to_ms(time_of_sequence,type_of_time=1)
            #labels = ['ID', 'latency','rep_stim_ID','coil_pos','coil_normal','coil_ori']
            if lines[line_index+5].split("\t") == ["\n"]: #the sequences we are interested in
                line_index += 1
                candidate_line_splitted = lines[line_index].split("\t")
                while True:
                    line_index += 1
                    if len(candidate_line_splitted) > 5:
                        if candidate_line_splitted[5] =='(ms)':
                            break
                    candidate_line_splitted = lines[line_index].split("\t")
                line_index += 1
                stimulation_line = lines[line_index].split("\t")
                while stimulation_line[0] != "\n": #go through all the stimulation lines in the current sequence and save information
                    if stimulation_line[12] != "-": #check if x-position is available, then everything regarding positions etc. should also be
                        trigger_info = {}
                        trigger_info['stimulus_id'] = stimulation_line[2]
                        trigger_info['repeated_stimulus_id'] = stimulation_line[9]
                        trigger_info['sequence_started'] = sequence_starts[-1]
                        trigger_info['recording_time_latency'] = int(stimulation_line[3])
                        trigger_info['recording_time_in_real_world'] = time_of_sequence_in_ms + trigger_info['recording_time_latency']
                        start_of_coil_pos = 12
                        trigger_info['coil_pos'] = np.array([np.float64(stimulation_line[start_of_coil_pos+i]) for i in range(3)])
                        start_of_coil_normal = 15
                        trigger_info['coil_normal'] = np.array([np.float64(stimulation_line[start_of_coil_normal+i]) for i in range(3)])
                        start_of_coil_dir = 18
                        trigger_info['coil_dir'] = np.array([np.float64(stimulation_line[start_of_coil_dir+i]) for i in range(3)])
                        trigger_markers.append(trigger_info)

                    #advance to the next line
                    line_index += 1
                    stimulation_line = lines[line_index].split("\t")
            else:
                line_index += 1
        elif "Motor Threshold Exam Starting stimulus:" in line:
            target_pos_mte = line.split(" ")[-1].strip("\n")
            print(target_pos_mte)
            line_index += 1
        else:
            line_index += 1


    # the target is the one that has been repeated the most (in here, we only had one target for ~1200 stims at the primary motor cortex)
    repeated_stimulus_ids = [trigger['repeated_stimulus_id'] for trigger in trigger_markers]
    stimulus_counts = Counter(repeated_stimulus_ids)
    most_common_repeated_stimulus = stimulus_counts.most_common(1)[0]
    most_common_repeated_stimulus_id = most_common_repeated_stimulus[0]
    #most_common_repeated_stimulus_count = most_common_repeated_stimulus[1]
    print(stimulus_counts)
    if target_pos_mte is not False:
        if target_pos_mte != most_common_repeated_stimulus_id:
            print(f"pos_id: target pos in rmt definition {target_pos_mte} is different than most common repeated stim {most_common_repeated_stimulus_id}")
            try:
                target_marker_most_common = [trigger_info for trigger_info in trigger_markers if trigger_info['stimulus_id'] == most_common_repeated_stimulus_id][0]
                target_marker_mte = [trigger_info for trigger_info in trigger_markers if trigger_info['stimulus_id'] == target_pos_mte][0]
                print(f"target pos: target pos rmt definition {target_marker_mte['coil_pos']} is different than most common repeated stim {target_marker_most_common['coil_pos']}")
            except IndexError:
                print("most_common_repeated_stimulus_id is not a targeted stimulation (no spatial information available)")
                return False
    # get the target marker
    try:
        target_marker = [trigger_info for trigger_info in trigger_markers if trigger_info['stimulus_id'] == most_common_repeated_stimulus_id][0]
        #print(target_marker)
        targeted_triggers = []
        for trigger_marker in trigger_markers:
            if trigger_marker['repeated_stimulus_id'] == target_marker['stimulus_id']: #only take the targeted stimuli
                #trigger_marker = calculate_pcd_and_dir_diff(trigger_marker, target_marker) #calculate the pcd_pos and the direction difference
                targeted_triggers.append(trigger_marker)

        information = {'trigger_markers':targeted_triggers, 'target_marker':target_marker, 'sequence_starting_times':sequence_starts}
        return information
    except IndexError:
        print("most_common_repeated_stimulus_id is not a targeted stimulation (no spatial information available)")
        return False
    

    


In [ ]:
filepath = rf"D:\REFTEP_ALL\Neuronavigation_nexstim_localite\Tuebingen_localite/" #has folders in a structure like subject1, subject2,.., subject_n'
filepath_stims = os.path.join(filepath, "stimulation_times")
os.makedirs(filepath_stims,exist_ok=True)
for subject in os.listdir(filepath):
    if "sub-" in subject: #if there are other files then do not look at them.
        print(subject)
        subject_localite_path = os.path.join(filepath, subject) #the path of the subject in the localite directory
        subject_path_stims = os.path.join(filepath_stims, subject)
        #os.makedirs(subject_path_stims,exist_ok=True)
        #get the path of the trigger marker file. This file is the LARGEST that exists in .../subject_localite_path/TMSTrigger/ that is in the group of files with the latest date (year, month, day).
        #Even though we only used one coil, trigger marker files for both Coil0 and Coil1 have the same date, but one of them is larger.
        trigger_markers_filepaths = get_trigger_marker_filepath(subject_localite_path)
        next_id = 0
        if trigger_markers_filepaths:
            for ind, trigger_marker_filepath in enumerate(trigger_markers_filepaths):
                next_id += 1
                #get the path of the instrument marker file. This file is the MOST RECENT file that exists in .../subject_localite_path/InstrumentMarkers/. This is the one that was most recently updated.
                #This file should never change after the actual stimulation is started.
                instrument_marker_filepath = get_instrument_marker_filepath(subject_localite_path)
                if subject == "sub-035":
                    constrain_recording_time = True
                    instrument_marker_filepath = r"D:\REFTEP_ALL\Neuronavigation_nexstim_localite\Tuebingen_localite\sub-035\InstrumentMarkers\InstrumentMarker20230328105137090.xml"
                else:
                    constrain_recording_time = True
                information_new = get_trigger_marker_info_xml(trigger_marker_filepath, instrument_marker_filepath, constrain_recording_time=constrain_recording_time, start_id=next_id) #get the information on trigger markers
                if ind > 0:
                    for key in ['trigger_markers']:
                        information[key] = information[key] + information_new[key]
                else:
                    information = information_new
                next_id = len(information['trigger_markers'])
            print(len(information['trigger_markers']))
            if len(information['trigger_markers']) != len(np.unique([trig['stimulus_id'] for trig in information['trigger_markers']])):
                print("bad stim ids")
            np.save(f'{subject_path_stims}/{subject}_stimulations',information)
            scipy.io.savemat(f'{subject_path_stims}/{subject}_stimulations.mat',information)
            #for trigm in information['trigger_markers']:
                #print(trigm['coil_pos'],information['instrument_marker']['coil_pos'])
        print("\n")

In [ ]:
filepath = rf"D:\REFTEP_ALL\Neuronavigation_nexstim_localite\Aalto_nexstim/" #has folders in a structure like subject1, subject2,.., subject_n
filepath_stims = os.path.join(filepath, "stimulation_times")
os.makedirs(filepath_stims,exist_ok=True)
for subject in os.listdir(filepath):
    if "REFTEP" in subject: #if there are other files then do not look at them.
        subject_directory_path = os.path.join(filepath, subject)
        files_in_dir = os.listdir(subject_directory_path)
        nbe_files = [file for file in files_in_dir if file.endswith(".nbe")]
        subject_path_stims = os.path.join(filepath_stims, subject)
        os.makedirs(subject_path_stims,exist_ok=True)
        if len(nbe_files) > 1:
            print(f"More than one .nbe file in directory for {subject}")
            break
        else:
            nbe_file = nbe_files[0]
        nbe_filepath = os.path.join(subject_directory_path,nbe_file)
        information = get_trigger_marker_info_nbe(nbe_filepath,subject)
        n_triggers = len(information['trigger_markers'])
        if n_triggers >= 1200:
            print(f'{subject} has {n_triggers} targeted stimuli at the most common target. Can include noise masking trials.')
        else:
            print(f'{subject} has {n_triggers} targeted stimuli at the most common target. Should be at least 1200! Check this!')
        np.save(f'{subject_path_stims}/{subject}_stimulations',information)
        scipy.io.savemat(f'{subject_path_stims}/{subject}_stimulations.mat',information)
        #print(information)